In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pandas as pd                                                   
from tensorflow.keras.models import Sequential, Model                        
from tensorflow.keras.layers import Dense, Dropout, LSTM, GRU, RepeatVector, TimeDistributed, Input               
from sklearn.preprocessing import MinMaxScaler                        
import numpy as np                                                    
import matplotlib.pyplot as plt                                        
import pickle
from scipy.stats import pearsonr

In [2]:
GV = pd.read_csv("PPI_GV.csv")

Material_comp = pd.DataFrame()

GV = GV.drop(["DATE"], axis = 1) 
GV = GV.iloc[::-1]
GV = GV.iloc[:393]
GV = GV.iloc[::-1]
GV = GV.reset_index(drop=True) 

CB = pd.read_csv("PPI_CB.csv")
CB = CB.drop(["DATE"], axis = 1) 
CB = CB.iloc[::-1]
CB = CB.iloc[:393]
CB = CB.iloc[::-1]
CB = CB.reset_index(drop=True) 

METAL = pd.read_csv("PPI_METAL.csv")
METAL = METAL.drop(["DATE"], axis = 1) 
METAL = METAL.iloc[::-1]
METAL = METAL.iloc[:393]
METAL = METAL.iloc[::-1]
METAL = METAL.reset_index(drop=True) 

CNC = pd.read_csv("PPI_CNC.csv")
CNC = CNC.drop(["DATE"], axis = 1) 
CNC = CNC.iloc[::-1]
CNC = CNC.iloc[:393]
CNC = CNC.iloc[::-1]
CNC = CNC.reset_index(drop=True) 

PL = pd.read_csv("PPI_PL.csv")
PL = PL.drop(["DATE"], axis = 1) 
PL = PL.iloc[::-1]
PL = PL.iloc[:393]
PL = PL.iloc[::-1]
PL = PL.reset_index(drop=True)

In [3]:
Material_comp["Gravel"] = GV
Material_comp["Conc_Brick"] = CB
Material_comp["Metals"] = METAL
Material_comp["Concrete_Mix"] = CNC
Material_comp["Plywood"] = PL

In [4]:
train = Material_comp.iloc[:354]
val = Material_comp.iloc[:354]

In [5]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled = scaler.fit_transform(Material_comp.values)

In [6]:
trainX = []
trainY = []

n_future = 1   
n_past = 14  

for i in range(n_past, len(scaled) - n_future +1):
    trainX.append(scaled[i - n_past:i, 0:train.shape[1]])
    trainY.append(scaled[i + n_future - 1:i + n_future, 0])

trainX, trainY = np.array(trainX,dtype=float), np.array(trainY,dtype=float)

In [13]:
trainY.shape

(379, 1)

In [23]:
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(GRU(64, activation='relu', return_sequences=True, recurrent_dropout=0.2))
model.add(LSTM(128, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(GRU(128, activation='relu', return_sequences=True, recurrent_dropout=0.2))
model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(GRU(64, activation='relu', return_sequences=False, recurrent_dropout=0.2))
model.add(Dense(trainX.shape[2]))
model.compile(optimizer='adam', loss=['mse','mape'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 14, 64)            17920     
                                                                 
 gru_18 (GRU)                (None, 14, 64)            24960     
                                                                 
 lstm_19 (LSTM)              (None, 14, 128)           98816     
                                                                 
 gru_19 (GRU)                (None, 14, 128)           99072     
                                                                 
 lstm_20 (LSTM)              (None, 14, 64)            49408     
                                                                 
 gru_20 (GRU)                (None, 64)                24960     
                                                                 
 dense_6 (Dense)             (None, 5)                

In [24]:
history = model.fit(trainX, trainY, epochs=50, batch_size=16, validation_split=0.1, verbose=1)

Epoch 1/50
22/22 [==============================] - 9s 80ms/step - loss: 0.0761 - val_loss: 0.0271
Epoch 2/50
22/22 [==============================] - 1s 47ms/step - loss: 0.0059 - val_loss: 0.0540
Epoch 3/50
22/22 [==============================] - 1s 40ms/step - loss: 0.0017 - val_loss: 0.0353
Epoch 4/50
22/22 [==============================] - 1s 39ms/step - loss: 7.0420e-04 - val_loss: 0.0048
Epoch 5/50
22/22 [==============================] - 1s 41ms/step - loss: 2.9072e-04 - val_loss: 0.0040
Epoch 6/50
22/22 [==============================] - 1s 46ms/step - loss: 2.2318e-04 - val_loss: 0.0040
Epoch 7/50
22/22 [==============================] - 1s 43ms/step - loss: 1.9167e-04 - val_loss: 0.0031
Epoch 8/50
22/22 [==============================] - 1s 46ms/step - loss: 1.9634e-04 - val_loss: 0.0038
Epoch 9/50
22/22 [==============================] - 1s 40ms/step - loss: 2.4498e-04 - val_loss: 3.6173e-04
Epoch 10/50
22/22 [==============================] - 1s 40ms/step - loss: 2.1922e

In [25]:
n_past = 16
n_days_for_prediction=7
prediction = model.predict(trainX[-n_days_for_prediction:]) #shape = (n, 1) where n is the n_days_for_prediction

prediction
scaled_back = scaler.inverse_transform(prediction)

1/1 [==============================] - 1s 998ms/step


In [14]:
from darts import metrics
rmse = metrics.metrics.rmse(trainY, scaled_back, intersect=True)
mse = metrics.metrics.mse(trainY, scaled_back, intersect=True)
mae = metrics.metrics.mae(trainY, scaled_back, intersect=True)
mape = metrics.metrics.mape(trainY, scaled_back, intersect=True)
r2 = metrics.metrics.r2_score(trainY, scaled_back, intersect=True)

AttributeError: 'numpy.ndarray' object has no attribute 'width'

In [12]:
scaled_back

array([[405.38217, 296.83423, 360.03958, 321.04306, 446.46783],
       [407.9328 , 298.4048 , 362.3515 , 323.03   , 449.8079 ],
       [410.21848, 299.8098 , 364.42358, 324.8087 , 452.81314],
       [412.41595, 301.1509 , 366.39273, 326.499  , 455.65295],
       [415.02545, 302.73615, 368.7003 , 328.4837 , 458.95197],
       [417.67142, 304.33508, 370.99933, 330.46262, 462.18344],
       [421.4104 , 306.59103, 374.22644, 333.246  , 466.6886 ]],
      dtype=float32)

In [ ]:
Material_comp

array([[0.01545016],
       [0.01610761],
       [0.01643634],
       [0.01610761],
       [0.01643634],
       [0.01742252],
       [0.01775125],
       [0.01742252],
       [0.01775125],
       [0.01742252],
       [0.01807997],
       [0.01873743],
       [0.02235342],
       [0.02268215],
       [0.02268215],
       [0.02268215],
       [0.02366833],
       [0.02399705],
       [0.02399705],
       [0.02399705],
       [0.02399705],
       [0.02531196],
       [0.02958541],
       [0.02925668],
       [0.03024286],
       [0.03254395],
       [0.03122904],
       [0.03254395],
       [0.0332014 ],
       [0.0332014 ],
       [0.03615995],
       [0.04043339],
       [0.03813231],
       [0.03878976],
       [0.04273448],
       [0.04339193],
       [0.04437811],
       [0.04700793],
       [0.0453643 ],
       [0.04635048],
       [0.04733666],
       [0.04799411],
       [0.04799411],
       [0.04865156],
       [0.04865156],
       [0.04898029],
       [0.05456864],
       [0.055